# MPS Library

## Organization

The components of the library are as follows (*italic* means "to be done")

1. Matrix Product State object

   a. [State classes](File%201a%20-%20MPS%20class.ipynb)

   b. [Sample states: W states, GHZ, AKLT, etc.](File%201b%20-%20MPS%20sample%20states.ipynb)
   
   c. [Canonical form](File%201c%20-%20Canonical%20form.ipynb)
   
   d. [Matrix Product Operators](File%201d%20-%20Matrix%20Product%20Operators.ipynb)

2. Ways to extract information about a state

   a. [Expectation values and correlations](File%202a%20-%20Expectation%20values.ipynb)
   
   b. *Density matrices*
   
   c. [Hamiltonians and operators](File%203c%20-%20Hamiltonians%20and%20operators.ipynb)

3. Solution of static problems.

   a. [Truncating an MPS](File%203a%20-%20Truncating%20an%20MPS.ipynb)
   
   b. *Ground state problems*
   
   c. *Stationary states of master equations*
   
4. Solution of time-evolution problems.

   a. *TEBD-like Trotter decompositions*
   
   b. *MPS-like Trotter decompositions*
   
   c. *Arnoldi / long-range methods*

5. MPS quantum register

   a. [Quantum Fourier Transform](File%205a%20-%20Quantum%20Fourier%20Transform.ipynb)

   b. [Algebraic operators](File%205b%20-%20Algebraic%20operators.ipynb)


The library can be read as a tutorial. A recommended reading order is 1a, 1b, 2a, 1c, 3a.

## Building

The library is designed as a series of more or less self-explaining notebooks, with Python code that can be executed or exported into a library. This choice of format allows for a more literate-like programming, combining images with equations and text, so as to make the algorithms more understandable.

In order to convert the notebooks into a library, we use a simple tool built by J. J. García Ripoll and available at [https://github.com/juanjosegarciaripoll/exportnb](GitHub). We download the latest version of the routine, which is not yet standard.

In [ ]:
import urllib.request
import os.path

if not os.path.isfile("exportnb.py"):
    urllib.request.urlretrieve("https://raw.githubusercontent.com/juanjosegarciaripoll/exportnb/master/exportnb.py",
                               "exportnb.py")

Once we have the export routine, we run through the notebooks in order, building a library that is to be stored in the `mps/` directory.

In [ ]:
import exportnb

In [ ]:
files = [
    'File 0 - Root.ipynb',
    'File 1a - MPS class.ipynb',
    'File 1b - MPS sample states.ipynb',
    'File 1c - Canonical form.ipynb',
    'File 1d - Matrix Product Operators.ipynb',
    'File 2a - Expectation values.ipynb',
    'File 3a - Truncating an MPS.ipynb' ,
    'File 3c - Hamiltonians and operators.ipynb' ,
    #'File 2 - Matrix Product Operators.ipynb',
    #'File 2a - Nearest neighbor Hamiltonian.ipynb',
    'File 4a - Time evolution - TEBD.ipynb',
    'File 5a - Quantum Fourier Transform.ipynb'
]

In [ ]:
exportnb.export_notebooks(files)

## Library core

The following is the glue code that loads all of the library components when we use `import mps`. It re-exports some commonly used functions and constants and sets some global variables that are used for debugging.

In [ ]:
# file: mps/__init__.py

import numpy as np
import scipy.sparse as sp

__all__ = ['state','expectation']

Note how we follow the literate programing conventions of the `exportnb` library, found in [GitHub](https://github.com/juanjosegarciaripoll/exportnb). The code box is labeled with the `# file:` comment indicating where this text will be exported to. Multiple boxes can be exported to the same or different files, based on these tags.

In [ ]:
# file: mps/tools.py
import numpy as np
import scipy.sparse as sp
from math import cos, sin, pi

def take_from_list(O, i):
    if type(O) == list:
        return O[i]
    else:
        return O

In [ ]:
# file: mps/tools.py

DEBUG = False

def log(*args):
    if DEBUG:
        print(*args)

In [ ]:
# file: mps/tools.py

def random_isometry(N, M=None):
    if M is None:
        M = N
    U = np.random.rand(N, M)
    U, s, V = np.linalg.svd(U, full_matrices=False)
    if M <= N:
        return U
    else:
        return V       
    

In [ ]:
# file: mps/tools.py


σx = np.array([[0.0, 1.0], [1.0, 0.0]])
σz = np.array([[1.0, 0.0], [0.0, -1.0]])
σy = -1j * σz @ σx


def random_Pauli():
    r = np.random.rand(2)
    θ = (2*r[0]-1) * np.pi
    ϕ = r[1] * np.pi
    return cos(ϕ) * (cos(θ) * σx + sin(θ) * σy) + sin(ϕ) * σz

In [ ]:
# file: mps/tools.py

def creation(d):
    # Returns d dimensional cration operator
    return np.diag(np.sqrt(np.arange(1,d)),-1).astype(complex)

def annihilation(d):
    # Returns d dimensional cration operator
    return np.diag(np.sqrt(np.arange(1,d)),1).astype(complex)

---
# Tests

In [ ]:
# file: mps/test/tools.py
import numpy as np
import scipy.sparse as sp

def similar(A, B, **kwdargs):
    if sp.issparse(A):
        A = A.todense()
    if sp.issparse(B):
        B = B.todense()
    return (A.shape == B.shape) & np.all(np.isclose(A, B, **kwdargs))

In [ ]:
# file: mps/test/tools.py


def almostIdentity(L, places=7):
    return np.all(np.isclose(L, np.eye(L.shape[0]), atol=10**(-places)))


def almostIsometry(A, places=7):
    N, M = A.shape
    if M < N:
        A = A.T.conj() @ A
    else:
        A = A @ A.T.conj()
    return almostIdentity(A, places=places)


def approximateIsometry(A, direction, places=7):
    if direction > 0:
        a, i, b = A.shape
        A = np.reshape(A, (a*i, b))
        C = A.T.conj() @ A
    else:
        b, i, a = A.shape
        A = np.reshape(A, (b, i*a))
        C = A @ A.T.conj()
    return almostIdentity(C)

In [ ]:
# file: mps/test/tools.py

import mps.state


def test_over_random_mps(function, d=2, N=10, D=10, repeats=10):
    for nqubits in range(1, N+1):
        for _ in range(repeats):
            function(mps.state.random(d, N, D))

In [ ]:
# file: mps/test/test_tools.py

import unittest
from mps.tools import *
from mps.test.tools import *

class TestTools(unittest.TestCase):
    
    def test_random_isometry(self):
        for N in range(1, 10):
            for M in range(1, 10):
                A = mps.tools.random_isometry(N, M)
                self.assertTrue(almostIsometry(A))

    def test_random_Pauli(self):
        for N in range(100):
            σ = random_Pauli()
            self.assertTrue(almostIdentity(σ @ σ))
            self.assertTrue(np.sum(np.abs(σ.T.conj() - σ)) == 0)

In [ ]:
suite1 = unittest.TestLoader().loadTestsFromNames(['__main__.TestTools'])
unittest.TextTestRunner(verbosity=2).run(suite1);